<a id='1'></a>
# Import modules

In [1]:
import keras.backend as K
import os
import tensorflow as tf

Using TensorFlow backend.
/opt/conda/envs/faceswap-GAN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/envs/faceswap-GAN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/envs/faceswap-GAN/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/envs/faceswap-GAN/lib/pytho

<a id='4'></a>
# Model Configuration

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [3]:
K.set_learning_phase(0)

In [4]:
# Input/Output resolution
RESOLUTION = 256 # 64x64, 128x128, 256x256
assert (RESOLUTION % 64) == 0, "RESOLUTION should be 64, 128, 256"

In [5]:
# Architecture configuration
arch_config = {}
arch_config['IMAGE_SHAPE'] = (RESOLUTION, RESOLUTION, 3)
arch_config['use_self_attn'] = True
arch_config['norm'] = "instancenorm" # instancenorm, batchnorm, layernorm, groupnorm, none
arch_config['model_capacity'] = "standard" # standard, lite

<a id='5'></a>
# Define models

In [6]:
from networks.faceswap_gan_model import FaceswapGANModel

In [7]:
model = FaceswapGANModel(**arch_config)

<a id='6'></a>
# Load Model Weights

In [8]:
model.load_weights(path="/parent_dir/models/v7")

Model weights files are successfully loaded.


<a id='12'></a>
# Video Conversion

In [9]:
from converter.video_converter import VideoConverter
from detector.face_detector import MTCNNFaceDetector

In [10]:
mtcnn_weights_dir = "./mtcnn_weights/"

fd = MTCNNFaceDetector(sess=K.get_session(), model_path=mtcnn_weights_dir)
vc = VideoConverter()

In [11]:
vc.set_face_detector(fd)
vc.set_gan_model(model)

### Video conversion configuration


- `use_smoothed_bbox`: 
    - Boolean. Whether to enable smoothed bbox.
- `use_kalman_filter`: 
    - Boolean. Whether to enable Kalman filter.
- `use_auto_downscaling`:
    - Boolean. Whether to enable auto-downscaling in face detection (to prevent OOM error).
- `bbox_moving_avg_coef`: 
    - Float point between 0 and 1. Smoothing coef. used when use_kalman_filter is set False.
- `min_face_area`:
    - int x int. Minimum size of face. Detected faces smaller than min_face_area will not be transformed.
- `IMAGE_SHAPE`:
    - Input/Output resolution of the GAN model
- `kf_noise_coef`:
    - Float point. Increase by 10x if tracking is slow. Decrease by 1/10x if trakcing works fine but jitter occurs.
- `use_color_correction`: 
    - String of "adain", "adain_xyz", "hist_match", or "none". The color correction method to be applied.
- `detec_threshold`: 
    - Float point between 0 and 1. Decrease its value if faces are missed. Increase its value to reduce false positives.
- `roi_coverage`: 
    - Float point between 0 and 1 (exclusive). Center area of input images to be cropped (Suggested range: 0.85 ~ 0.95)
- `enhance`: 
    - Float point. A coef. for contrast enhancement in the region of alpha mask (Suggested range: 0. ~ 0.4)
- `output_type`: 
    - Layout format of output video: 1. [ result ], 2. [ source | result ], 3. [ source | result | mask ]
- `direction`: 
    - String of "AtoB" or "BtoA". Direction of face transformation.

In [33]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.IMAGE_SHAPE,
    # ===== Tunable =====
    "kf_noise_coef": 0.0023,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.7,
    "roi_coverage": 0.9,
    "enhance": 0,
    "output_type": 1,
    "direction": "AtoB",
}

# Start video conversion


- `input_fn`: 
    - String. Input video path.
- `output_fn`: 
    - String. Output video path.
- `duration`: 
    - None or a non-negative float tuple: (start_sec, end_sec). Duration of input video to be converted
    - e.g., setting `duration = (5, 7.5)` outputs a 2.5-sec-long video clip corresponding to 5s ~ 7.5s of the input video.

In [34]:
input_fn = "/parent_dir/videos/v7/1_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_1_0023.mp4"
duration = None

In [35]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/309 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_1_0023.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_1_0023.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_1_0023.mp4


In [36]:
input_fn = "/parent_dir/videos/v7/2_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_2_0023.mp4"
duration = None

In [37]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/349 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_2_0023.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_2_0023.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_2_0023.mp4


In [38]:
input_fn = "/parent_dir/videos/v7/3_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_3_0023.mp4"
duration = None

In [39]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/871 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_3_0023.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_3_0023.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_3_0023.mp4


In [40]:
options = {
    # ===== Fixed =====
    "use_smoothed_bbox": True,
    "use_kalman_filter": True,
    "use_auto_downscaling": False,
    "bbox_moving_avg_coef": 0.65,
    "min_face_area": 35 * 35,
    "IMAGE_SHAPE": model.IMAGE_SHAPE,
    # ===== Tunable =====
    "kf_noise_coef": 0.0022,
    "use_color_correction": "hist_match",
    "detec_threshold": 0.7,
    "roi_coverage": 0.9,
    "enhance": 0,
    "output_type": 1,
    "direction": "AtoB",
}

In [41]:
input_fn = "/parent_dir/videos/v7/1_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_1_0022.mp4"
duration = None

In [42]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/309 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_1_0022.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_1_0022.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_1_0022.mp4


In [43]:
input_fn = "/parent_dir/videos/v7/2_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_2_0022.mp4"
duration = None

In [44]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/349 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_2_0022.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_2_0022.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_2_0022.mp4


In [45]:
input_fn = "/parent_dir/videos/v7/3_resized.avi"
output_fn = "/parent_dir/conversion/v7/v7_kf_3_0022.mp4"
duration = None

In [46]:
vc.convert(input_fn=input_fn, output_fn=output_fn, options=options, duration=duration)

t:   0%|          | 0/871 [00:00<?, ?it/s, now=None]

Moviepy - Building video /parent_dir/conversion/v7/v7_kf_3_0022.mp4.
Moviepy - Writing video /parent_dir/conversion/v7/v7_kf_3_0022.mp4



Moviepy - Done !
Moviepy - video ready /parent_dir/conversion/v7/v7_kf_3_0022.mp4
